### *Data Notebook for* 
# The BioFragment Database (BFDb): An Open-Data Platform for Computational Chemistry Analysis of Noncovalent Interactions
### Burns, Faver, Zheng, Marshall, Smith, Vanommeslaeghe, MacKerell, Merz, and Sherrill

## Configuration

##### Prerequisites (satisfied by installer)
* Python 2.7
* Jupyter
* matplotlib
* qcdb (adjust import path below if have separate git repository)

##### Optional Prerequisites (NOT satisfied by installer)

In [ ]:
# command to open pdf files. only needed for tables and standalone=True. below works for Mac.
pdfviewer = 'open /Applications/Preview.app'

# command to run LaTeX. only needed for tables and standalone=True. below works for Mac w/TeXShop
pdflatex = '/usr/texbin/pdflatex'

In [ ]:
import os
import errno
%matplotlib inline
import sys

In [ ]:
# if _not_ running from installer and qcdb _not_ in :envvar:`PYTHONPATH`, put path to repo here
#sys.path.insert(1, '/Users/loriab/linux/qcdb')
import qcdb
qcdb.__file__

##### Change these before running

In [ ]:
# existing, writable, empty (to start with) project directory for notebook to write to
wdir = '/opt/anaconda1anaconda2anaconda3/bfdb_analysis'
%cd {wdir}

##### Table Graphics Configuration: Uncomment one of three
_Background_: the tables have a lot of little figures that must be present for LaTeX to compile but which are tedious to generate.
_Recommendation_: These little figures come pre-built with the installer so easiest to just use them. If you want to rebuild, uncomment the first set, execute the block, run section IV that generates all figures any of the tables need, then recomment the first set and uncomment the second set, and execute this block again. Thereafter, you can run any table sections w/o regenerating figures.

In [ ]:
# run from jupyter, pop up tables, fresh build figures
# plotpath = 'autogen'
# standalone = True

# run from jupyter, pop up tables, reuse figures
# plotpath = ''
# standalone = True

# run from jupyter, prep tables tex for paper/suppmat, reuse figures
plotpath = '/opt/anaconda1anaconda2anaconda3/bfdb_analysis/'
standalone = False

##### Tables Configuration

In [ ]:
# plotting area btwn +/- tblxlimit kcal/mol for thread plots in tables
tblxlimit = 4.0

# guide lines at +/- tblxlines kcal/mol values for thread plots in tables
tblxlines = [0.0, 0.3, 1.0, 4.0]

# maximum color at +/- tblialimit kcal/mol for mini Iowa plots in tables
tblialimit = 1.0

# whether to plot incomplete modelchems in tables. safe b/c footnotes created if any datum missing
tblfailoninc = False

# summary error statistic in tables is MAE, mean absolute error
tblerr = ['mae']

## Loading SSI & BBI databases and QC results

In [ ]:
from qcdb.dbwrap import oxcom, fnreservoir
from qcdb import textables  # to define tableplans

In [ ]:
# `loadfrompickle=True` signals to load data from {qcdb-top-dir}/data/{database}*pickle files.
#   this is far faster than reloading the data from python command files "/{database}*py
bfdb = qcdb.Database(['ssi', 'bbi'], loadfrompickle=True)
ssi = qcdb.Database('ssi', loadfrompickle=True)
bbi = qcdb.Database('bbi', loadfrompickle=True)
print bfdb.available_projects()
for pj in bfdb.available_projects():
    bfdb.load_qcdata_byproject(pj)
    ssi.load_qcdata_byproject(pj)
    bbi.load_qcdata_byproject(pj)
bfdb.promote_Subset()

## I. Iowa plots (Figs. 4 & 5)

In [ ]:
# isolate in a directory
subdir = 'fig_iowa_array'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
# maximum color at +/- xlimit kcal/mol
xlimit = 1.0

# files only. turn off plotting within notebook b/c colors don't show through layers.
view = False

# model chemistries to plot
dbmc = [
    'GAFF--na',  # a
    'CGENFF--na',  # b
    'AM1--na',  # c
    'PM6DH2--na',  # d
    'PBEH3C-dfhf_unCP-def2msvp',  # e

    'M052X-dfhf_unCP-adz',  # f
    'WB97XV-unCP-atz',  # g
    'WB97MV-unCP-atz',  # h
    'PBE0D3MBJ-dfhf_CP-adz',  # i
    'B3LYPD3MBJ-dfhf_CP-adz',  # j
    'B2PLYPD3M-dfhf_dfmp_CP-atz',  # k
    
    'MP2-dfhf_dfmp_CP-atqz',  # l
    'SCSMIMP2-dfhf_dfmp_CP-atz',  # m
    'DWMP2-dfhf_dfmp_CP-adtz',  # n
    'SAPT2P-SA-adz',  # o
    'MP2CF12-SA-adz',  # p
    ]

In [ ]:
for mc in dbmc:
    print mc
    ssi.plot_iowa(mc, graphicsformat=['pdf'], failoninc=False, view=view, xlimit=xlimit)

In [ ]:
%ls

## II. Metal benchmarks thread plots (Fig. 3)

In [ ]:
# isolate in a directory
subdir = 'fig_metals'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
db4 = qcdb.DB4(loadfrompickle=True)  # S22, NBC10, HBC6, HSG
db4.load_qcdata_byproject('pt2')  # JCP 141 234111 (2014) (doi: 10.1063/1.4903765)

In [ ]:
db4.plot_modelchems(['C2011BENCH', 'DWCCSDTF12-CP-adz', 'MP2CF12-CP-adz'], xlimit=2.0, sset='tt-5min')
db4.plot_modelchems(['C2011BENCH', 'DWCCSDTF12-CP-adz', 'MP2CF12-CP-adz'], labeled=False, xlimit=2.0, sset='tt-5min')

In [ ]:
for ss in bfdb.sset.keys():
    print ss
    bfdb.plot_modelchems(['SAPT0-SA-jadz', 'SAPT0S-SA-jadz', 'SAPT2P-SA-adz'], labeled=True, sset=ss)

In [ ]:
%ls
del db4

## III. Ternary plots (Fig. 2)

In [ ]:
s22 = qcdb.Database('s22', loadfrompickle=True)

In [ ]:
# isolate in a directory
subdir = 'fig_ternary_mpl'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
# graphics formats
gf = ['pdf', 'png']

# SSI, SSI500, SSI100, and other subsets
ssisss = ['default', 'ssi500', 'ssi100']

# BBI, BBI25, and other subsets
bbisss = ['default', 'bbi25', 'shb', 'ua']

# other (full) databases
dbs = ['hsg', 'ubq', 's22by5',
       #'s22', 'nbc10', 'hbc6', 's22by7', 'achc', 'a24', 's66', 'jsch', 'nbc10ext'  # not used in paper but available
      ]

In [ ]:
for ss in ssisss:
    print 'SSI', ss
    ssi.plot_ternary(graphicsformat=gf, sset=ss, labeled=False)
    ssi.plot_ternary(graphicsformat=gf, sset=ss, labeled=True)

for ss in bbisss:
    print 'BBI', ss
    bbi.plot_ternary(graphicsformat=gf, sset=ss, labeled=False)
    bbi.plot_ternary(graphicsformat=gf, sset=ss, labeled=True)

In [ ]:
%ls

## IV. Primary SuppMat Tables (Tables S-5 through S-19) 
##### basis-major w/Iowa & thread: 1 MM, 2 WFN, 12 DFT

In [ ]:
# isolate in a directory
subdir = 'tbl_modelchems'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
def table_bfdb_suppmat(**kwargs):
    """Specialization of table_generic into table with as many statistics
    as will fit plus embedded Iowa and thread
    diagram as suitable for supplementary material. Multiple tables are
    formed, one for each in *bas* with lines *mtd* within each table.

    """
    rowplan = ['bas', 'mtd']
    columnplan = [
        ['l', r"""Method \& Basis Set""", '', textables.label, {}],
        ['d', 'BBI', 'SHB', textables.val, {'sset': 'shb', 'dbse': 'BBI'}],
        ['d', 'BBI', 'UA', textables.val, {'sset': 'ua', 'dbse': 'BBI'}],
        ['d', 'BBI', '25', textables.val, {'sset': 'bbi25', 'dbse': 'BBI'}],
        ['d', 'BBI', 'TT', textables.val, {'sset': 'default', 'dbse': 'BBI'}],
        ['d', 'SSI', r"""$\bm{+/+}$""", textables.val, {'sset': 'pospos', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-/-}$""", textables.val, {'sset': 'negneg', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+/-}$""", textables.val, {'sset': 'posneg', 'dbse': 'SSI'}],
        ['d', 'SSI', 'pl/pl', textables.val, {'sset': 'polarpolar', 'dbse': 'SSI'}],
        ['d', 'SSI', 'al/al', textables.val, {'sset': 'aliphaliph', 'dbse': 'SSI'}],
        ['d', 'SSI', 'ar/ar', textables.val, {'sset': 'arylaryl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'al/ar', textables.val, {'sset': 'alipharyl', 'dbse': 'SSI'}],
        ['d', 'SSI', '100', textables.val, {'sset': 'ssi100', 'dbse': 'SSI'}],
        ['d', 'SSI', '500', textables.val, {'sset': 'ssi500', 'dbse': 'SSI'}],
        ['d', 'SSI', 'TT', textables.val, {'sset': 'default', 'dbse': 'SSI'}],
        ['c', r"""Iowa\footnotemark[1]""", '', textables.liliowa, {}],
        ['l', r"""Error Distribution\footnotemark[2]""", '', textables.flat, {'sset': 'ssi'}]
    ]

    footnotes = [fnreservoir['liliowa'].format('SSI ', kwargs['ialimit']), 
                 fnreservoir['flat'].format('SSI ', kwargs['xlimit'], oxcom(kwargs['xlines']))]
    landscape = True
    theme = 'si1bfdbmc'
    title = r"""Interaction energy (kcal/mol) {{err}} subset statistics computed with {{opt}}{0}.""".format(
                '' if kwargs['subjoin'] else r""" and {bas}""")
    return rowplan, columnplan, landscape, footnotes, title, theme

In [ ]:
tblfile = 'si1mm'
def make_bfdb_Tables_S1mm(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    dbobj.table_wrapper(mtd=['GAFF', 'CGENFF', 'AM1', 'PM6DH2', 'PBEH3C'],
                        bas=['na', 'def2msvp'],
                        opt=[''],
                        opttarget={'default': ['', 'dfhf_unCP']},
                        subjoin=True,
                        tableplan=table_bfdb_suppmat,
                        filename=tblfile,
                        title=r"""Interaction energy (kcal/mol) MAE statistics by force-field and semi-empirical methods.""",
                        theme='si1bfdbmc-mm',
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

make_bfdb_Tables_S1mm(bfdb)
if standalone:
    !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}

In [ ]:
tblfile = 'si1wfn1'
def make_bfdb_Tables_S1wfn(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    dbobj.table_wrapper(mtd=['SAPT2P',
                             'MP2', 'SCSMP2', 'SCSNMP2', 'SCSMIMP2', 'DWMP2', 'MP2C',
                             'MP2F12', 'SCSMP2F12', 'SCSNMP2F12', 'SCSMIMP2F12', 'DWMP2F12', 'MP2CF12',
                             'CCSDAF12', 'CCSDBF12', 'SCSCCSDAF12', 'SCSCCSDBF12', 'SCMICCSDAF12', 'SCMICCSDBF12', 
                             'CCSDTAF12', 'CCSDTBF12', 'DWCCSDTF12'],
                        bas=['adz'],                       
                        opt=[''],
                        opttarget={'default': ['CP', 'dfhf_dfmp_CP', 'SA', 'dfhf_dfmp_SA']},
                        subjoin=False,
                        suppressblanks=False,
                        tableplan=table_bfdb_suppmat,
                        filename=tblfile,
                        title=r"""Interaction energy (kcal/mol) {err} statistics by wavefunction methods with aug-cc-pVDZ, CP-corrected.""",
                        theme='si1bfdbmc-wfn1',
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

make_bfdb_Tables_S1wfn(bfdb)
if standalone:
    !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}

In [ ]:
def make_bfdb_Tables_S1wfn2(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    tblfile = 'si1wfn2'
    dbobj.table_wrapper(mtd=['MP2', 'SCSMP2', 'SCSNMP2', 'SCSMIMP2', 'DWMP2', 'MP2C'],
                        bas=['atz', 'adtz', 'qz', 'aqz', 'atqz', 'atqzadz'],                        
                        opt=[''],
                        opttarget={'default': ['CP', 'dfhf_dfmp_CP', 'SA', 'dfhf_dfmp_SA']},
                        subjoin=True,                        
                        suppressblanks=True,
                        tableplan=table_bfdb_suppmat,
                        filename=tblfile,
                        title=r"""Interaction energy (kcal/mol) {err} statistics by wavefunction methods with larger basis sets, CP-corrected.""",
                        theme='si1bfdbmc-wfn2',
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

    if standalone:
        !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}
        
make_bfdb_Tables_S1wfn2(bfdb)

In [ ]:
def make_bfdb_Tables_S1dft(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    tblfile = 'si1dft'
    dbobj.table_wrapper(mtd=[ 
                             'PBE',    'PBED2',    'PBED3',
                             'BP86',   'BP86D2',   'BP86D3',
                             'BLYP',   'BLYPD2',   'BLYPD3',
                             'B97',    'B97D2',    'B97D3',
                             'PBE0',   'PBE0D2',   'PBE0D3',
                             'B3LYP',  'B3LYPD2',  'B3LYPD3',
                             'WPBE',               'WPBED3',
                             'B2PLYP', 'B2PLYPD2', 'B2PLYPD3',
                            ],
                        bas=['adz', 'atz', 'def2qzvp'],
                        opt=['CP', 'unCP'],
                        opttarget={'default': ['', 'dfhf', 'dfmp_dfhf']},
                        subjoin=False,
                        suppressblanks=True,
                        tableplan=table_bfdb_suppmat,
                        filename=tblfile,
                        title=r"""Interaction energy (kcal/mol) {err} statistics by DFT methods computed with {opt} and {bas}.""",
                        theme='si1bfdbmc-dft',
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

    if standalone:
        !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}
        
make_bfdb_Tables_S1dft(bfdb)

In [ ]:
def make_bfdb_Tables_S1dftd3(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    tblfile = 'si1dftd3'
    dbobj.table_wrapper(mtd=[                         
                             'PBED3',    'PBED3M',    'PBED3BJ',    'PBED3MBJ',
                             'BP86D3',   'BP86D3M',   'BP86D3BJ',   'BP86D3MBJ',
                             'BLYPD3',   'BLYPD3M',   'BLYPD3BJ',   'BLYPD3MBJ',
                             'B97D3',    'B97D3M',    'B97D3BJ',    'B97D3MBJ',
                             'PBE0D3',   'PBE0D3M',   'PBE0D3BJ',   'PBE0D3MBJ',
                             'B3LYPD3',  'B3LYPD3M',  'B3LYPD3BJ',  'B3LYPD3MBJ',
                             'M052X', 'WB97XD', 'WB97XV', 'WB97MV',
                             'WPBED3',   'WPBED3M',   'WPBED3BJ',   'WPBED3MBJ',
                             'B2PLYPD3', 'B2PLYPD3M', 'B2PLYPD3BJ', 'B2PLYPD3MBJ',            
                            ],
                        bas=['adz', 'atz', 'def2qzvp'],
                        opt=['CP', 'unCP'],
                        opttarget={'default': ['', 'dfhf', 'dfmp_dfhf']},
                        subjoin=False,
                        suppressblanks=True,
                        tableplan=table_bfdb_suppmat,
                        filename=tblfile,
                        title=r"""Interaction energy (kcal/mol) {err} statistics by DFT-D3 methods computed with {opt} and {bas}.""",
                        theme='si1bfdbmc-dftd3',
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

    if standalone:
        !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}
        
make_bfdb_Tables_S1dftd3(bfdb)

In [ ]:
def make_bfdb_Tables_S1sapt(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    tblfile = 'si1sapt'
    dbobj.table_wrapper(mtd=['SAPT0', 'SAPT0S', 'SAPT2', 'SAPT2P'],
                        bas=['jadz', 'adz'],                        
                        opt=[''],
                        opttarget={'default': ['SA']},
                        subjoin=True,                        
                        suppressblanks=True,
                        tableplan=table_bfdb_suppmat,
                        filename=tblfile,
                        title=r"""Interaction energy (kcal/mol) {err} statistics by SAPT methods.""",
                        theme='si1bfdbmc-sapt',
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

    if standalone:
        !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}
        
make_bfdb_Tables_S1sapt(bfdb)

## V. Primary Paper Tables (Tables II, III, IV)
##### basis-major w/Iowa & thread: 1 MM, 1 WFN, 1 DFT

In [ ]:
# isolate in a directory
subdir = 'tbl_modelchems'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
# extra subsets used in primary paper tables
bfdb.add_Subset_union('npblk', ['aliphaliph', 'arylaryl', 'alipharyl'])
bfdb.add_Subset_union('ionblk', ['pospos', 'negneg', 'posneg'])
bfdb.add_Subset_union('plslice', ['pospolar', 'negpolar', 'polarpolar', 'polaraliph', 'polararyl'])

In [ ]:
def table_bfdb_paper(**kwargs):
    """Specialization of table_generic into table with as many statistics
    as will fit plus embedded slat
    diagram as suitable for supplementary material. Multiple tables are
    formed, one for each in *bas* with lines *mtd* within each table.

    """
    if (tblxlines == [0.0, 0.3, 1.0, 4.0]) and plotpath:
        guidecell = """\includegraphics[width=6.67cm,height=3.5mm]{bfdb_analysis/tbl_modelchems/flat_0p0_0p0_1p0_4p0_lbld.pdf}"""
    else:
        guidecell = ''
    
    
    rowplan = ['bas', 'mtd']
    columnplan = [
        ['l', r"""Method \& Basis Set""", '', textables.label, {}],
        ['d', 'BBI', r"""TT\footnotemark[3]""", textables.val, {'sset': 'default', 'dbse': 'BBI'}],
        ['d', 'SSI', r"""ion\footnotemark[4]""", textables.val, {'sset': 'ionblk', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""pl\footnotemark[5]""", textables.val, {'sset': 'plslice', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""npl\footnotemark[6]""", textables.val, {'sset': 'npblk', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""TT\footnotemark[3]""", textables.val, {'sset': 'default', 'dbse': 'SSI'}],
        ['c', r"""Iowa\footnotemark[1]""", '', textables.liliowa, {}],
        ['l', r"""Error Distribution\footnotemark[2]""", guidecell, textables.flat, {'sset': 'ssi'}]
    ]

    footnotes = [fnreservoir['liliowa'].format('SSI ', kwargs['ialimit']), 
                 fnreservoir['flat'].format('SSI ', kwargs['xlimit'], oxcom(kwargs['xlines'])),
                 """Total database""",
                 """Charged block: union of $+/+$, $+/-$, $-/-$.""",
                 """Polar-containing: union of +/pl, $-$/pl, pl/pl, pl/al, pl/ar.""",
                 """Nonpolar block: union of al/al, al/ar, ar/ar."""]
    landscape = False
    theme = 'ppbfdbmc'
    title = r"""Interaction energy (kcal/mol) {{err}} subset statistics computed with {{opt}}{0}.""".format(
                '' if kwargs['subjoin'] else r""" and {bas}""")
    return rowplan, columnplan, landscape, footnotes, title, theme

In [ ]:
def make_bfdb_Tables_mm(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    tblfile = 'ppmm'
    dbobj.table_wrapper(mtd=['GAFF', 'CGENFF', 'AM1', 'PM6DH2', 'PBEH3C'],
                        bas=['na', 'def2msvp'],
                        opt=[''],
                        opttarget={'default': ['', 'dfhf_unCP']},
                        subjoin=True,
                        tableplan=table_bfdb_paper,
                        filename=tblfile,
                        title=r"""Interaction energy (kcal/mol) {err} statistics by force-field and semi-empirical methods.""",
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

    if standalone:
        !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}
        
make_bfdb_Tables_mm(bfdb)

# further adjustments for paper inclusion
# * remove "Basis Set" and "no applicable basis" and "def2-mSVP (3 lines)
# * remove hline and 5 empty lines (6 lines)
# * in Missing footnotes, change "reactions" to "interactions" (4 lines)
# * add \cite{gaffmissing} to first three Missing lines (3 lines)
# * and \cite{charmmmissing} to last three Missing lines (3 lines)
# * change label to "\label{tbl:qcdb-ppbfdbmc--na-mae}}" (1 line)

In [ ]:
def make_bfdb_Tables_wfn(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    tblfile = 'ppwfn'
    dbobj.table_wrapper(mtd=['SAPT2P',
                             'MP2', 'SCSMP2', 'SCSNMP2', 'SCSMIMP2', 'DWMP2', 'MP2C',
                             'MP2F12', 'SCSMP2F12', 'SCSNMP2F12', 'SCSMIMP2F12', 'DWMP2F12', 'MP2CF12',
                             'CCSDAF12', 'CCSDBF12', 'SCSCCSDAF12', 'SCSCCSDBF12', 'SCMICCSDAF12', 'SCMICCSDBF12', 
                             'CCSDTAF12', 'CCSDTBF12', 'DWCCSDTF12'],  # 'CCSDTNSAF12'
                        bas=['adz', 'atqzadz', 'atz', 'qz', 'aqz', 'atqz'],  # 'adtz'
                        opt=[''],
                        opttarget={'default': ['CP', 'dfhf_dfmp_CP', 'SA', 'dfhf_dfmp_SA']},
                        subjoin=True,
                        suppressblanks=True,
                        tableplan=table_bfdb_paper,
                        filename=tblfile,
                        title=r"""Interaction energy (kcal/mol) {err} statistics by wavefunction methods, CP-corrected.""",
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

    if standalone:
        !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}
        
make_bfdb_Tables_wfn(bfdb)

# further adjustments for paper inclusion
# * comment out SCS-CCSD-F12a/b (2 lines)
# * comment out all SCS(N)-MP2) for >aDZ (4 lines)
# * comment out SCS-MP2 & DW-MP2 for QZ
# * comment out SCS-MP2 & SCS(MI)-MP2 for aTQZ (2 lines)
# * add the citation to this line and the aTQZ (2 lines): \textbf{[aTQZ; $\delta$:aDZ]}\cite{bracketdelta} \\

In [ ]:
tblfile = 'ppdftm'
def make_bfdb_Tables_dft(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    dbobj.table_wrapper(mtd=['PBED3M', 'BP86D3M', 'BLYPD3MBJ', 'B97D3MBJ', 'PBE0D3M', 'B3LYPD3MBJ',
                             'M052X',
                             'WPBED3', 'WB97XD', 'WB97XV', 'WB97MV',
                             'B2PLYPD3M'],
                        bas=['adz', 'atz', 'def2qzvp'],
                        opt=['CP', 'unCP'],
                        opttarget={'default': ['', 'dfhf', 'dfmp_dfhf']},
                        subjoin=True,
                        suppressblanks=True,
                        tableplan=table_bfdb_paper,
                        filename=tblfile,                         
                        title=r"""Interaction energy (kcal/mol) {err} statistics by DFT methods.""",
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)

make_bfdb_Tables_dft(bfdb)
if standalone:
    !{pdflatex} {tblfile} && {pdfviewer} {tblfile + '.pdf'}

# further adjustments for paper inclusion
# * two tbls are formed. comment out qzvp of first tbl thru header of second tbl so only one tbl built (many lines)
# * add ', CP' to first basis blocks and ', unCP' to further basis blocks (6 lines)
# * arrange blocks adz-uncp, adz-cp, atz-uncp, atz-cp, qzvp-uncp (many lines)
# * comment out PBE and B97 in atz-cp and qzvp-uncp (4 lines)
# * add footnotetexts 7 & 8 'Missing 2--7 of 230 interactions' & 3380 (2 lines)
# * add \cite{gganoconv} to above footnotetexts (2 lines)
# * alter footnotemarks so all ion go with 7 and all SSI TT go with 8 (12 lines)

## VI. Secondary SuppMat Tables (Tables S-20 through S-39) 
##### method/functional family-major w/Iowa w/colored DFT: #1 MM, 1 WFN, 18 DFT

In [ ]:
# isolate in a directory
subdir = 'tbl_modelchems'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
def table_bfdb_allsset(**kwargs):
    """Specialization of table_generic into table with as many statistics
    as will fit plus embedded Iowa
    diagram as suitable for supplementary material. Multiple tables are
    formed, one for each in *bas* with lines *mtd* within each table.

    """
    rowplan = ['bas', 'opt', 'mtd']
    columnplan = [
        ['l', r"""Method \& Basis Set""", '', textables.label, {}],
        ['d', 'BBI', 'SHB', textables.val, {'sset': 'shb', 'dbse': 'BBI'}],
        ['d', 'BBI', 'UA', textables.val, {'sset': 'ua', 'dbse': 'BBI'}],
        ['d', 'BBI', 'TT', textables.val, {'sset': 'default', 'dbse': 'BBI'}],
        ['d', 'SSI', r"""$\bm{+/+}$""", textables.val, {'sset': 'pospos', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+/-}$""", textables.val, {'sset': 'posneg', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+}$/pl""", textables.val, {'sset': 'pospolar', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+}$/al""", textables.val, {'sset': 'posaliph', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+}$/ar""", textables.val, {'sset': 'posaryl', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-/-}$""", textables.val, {'sset': 'negneg', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-}$/pl""", textables.val, {'sset': 'negpolar', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-}$/al""", textables.val, {'sset': 'negaliph', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-}$/ar""", textables.val, {'sset': 'negaryl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'pl/pl', textables.val, {'sset': 'polarpolar', 'dbse': 'SSI'}],
        ['d', 'SSI', 'pl/al', textables.val, {'sset': 'polaraliph', 'dbse': 'SSI'}],
        ['d', 'SSI', 'pl/ar', textables.val, {'sset': 'polararyl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'al/al', textables.val, {'sset': 'aliphaliph', 'dbse': 'SSI'}],
        ['d', 'SSI', 'al/ar', textables.val, {'sset': 'alipharyl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'ar/ar', textables.val, {'sset': 'arylaryl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'TT', textables.val, {'sset': 'default', 'dbse': 'SSI'}],
        ['c', r"""Iowa\footnotemark[1]""", '', textables.liliowa, {}],
    ]

    footnotes = [fnreservoir['liliowa'].format('SSI ', kwargs['ialimit'])] 
    landscape = True
    theme = 'si2bfdbmc'
    title = r"""Interaction energy (kcal/mol) {{err}} subset statistics computed with {{opt}}{0}.""".format(
                '' if kwargs['subjoin'] else r""" and {bas}""")
    return rowplan, columnplan, landscape, footnotes, title, theme

In [ ]:
def table_bfdb_allssetmmwfn(**kwargs):
    """Specialization of table_generic into table with as many statistics
    as will fit plus embedded slat
    diagram as suitable for supplementary material. Multiple tables are
    formed, one for each in *bas* with lines *mtd* within each table.

    """
    rowplan, columnplan, landscape, footnotes, title, theme = table_bfdb_allsset(**kwargs)
    rowplan = ['mtd', 'bas']

    return rowplan, columnplan, landscape, footnotes, title, theme

In [ ]:
def make_bfdb_Tables_S2mm(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    tblname = 'si2mm'
    dbobj.table_wrapper(mtd=['GAFF', 'CGENFF', 'AM1', 'PM6DH2', 'PBEH3C'],
                        bas=['na', 'def2msvp'],
                        opt=[''],
                        opttarget={'default': ['', 'dfhf_unCP']},
                        subjoin=True,                        
                        suppressblanks=True,
                        tableplan=table_bfdb_allssetmmwfn,
                        filename=tblname,
                        title=r"""Interaction energy (kcal/mol) {err} subset statistics by force-field and semi-empirical methods.""",
                        theme='si2bfdbmc-mm',
                        err=tblerr,
                        failoninc=tblfailoninc,
                        xlimit=tblxlimit,
                        xlines=tblxlines,
                        ialimit=tblialimit,
                        plotpath=plotpath,
                        standalone=standalone)
    if standalone:
        !{pdflatex} {tblname} && {pdfviewer} {tblname + '.pdf'}

make_bfdb_Tables_S2mm(bfdb)

In [ ]:
def make_bfdb_Tables_S2wfn(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    tblname = 'si2wfn'
    dbobj.table_wrapper(mtd=['MP2', 'SCSMP2', 'SCSNMP2', 'SCSMIMP2', 'DWMP2'],
                            bas=['adz', 'atz', 'adtz', 'qz', 'aqz', 'atqz'],
                            opt=[''],
                            opttarget={'default': ['CP', 'dfhf_dfmp_CP', 'SA', 'dfhf_dfmp_SA']},
                            subjoin=True,                        
                            suppressblanks=True,
                            tableplan=table_bfdb_allssetmmwfn,
                            filename=tblname,
                            title=r"""Interaction energy (kcal/mol) {err} subset statistics by MP2 methods, CP-corrected.""",
                            theme='si2bfdbmc-wfn',
                            err=tblerr,
                            failoninc=tblfailoninc,
                            xlimit=tblxlimit,
                            xlines=tblxlines,
                            ialimit=tblialimit,
                            plotpath=plotpath,
                            standalone=standalone)
    if standalone:
        !{pdflatex} {tblname} && open /Applications/Preview.app {tblname + '.pdf'}

make_bfdb_Tables_S2wfn(bfdb)

In [ ]:
def colorval(best, better=0.15, good=0.25, rotten=0.5):
    def colorscheme(kw):
        brick = r"""\cellcolor[RGB]{153,0,0}"""
        paleorange = r"""\cellcolor[HTML]{FFCC99}"""
        paleblue = r"""\cellcolor[HTML]{CCE5FF}"""
        lilac = r"""\cellcolor[RGB]{204,153,255}"""
        mval = kw['matelem']
        if mval.isspace():
            return textables.val(kw)
        else:
            fval = float(mval)
            cellcolor = ''
            if fval > rotten:
                cellcolor = brick
            if fval <= good:
                cellcolor = paleorange
            if fval <= better:
                cellcolor = paleblue
            if fval <= best:
                cellcolor = lilac
            return cellcolor + textables.val(kw)
    return colorscheme


def table_bfdb_allssetwcolor(**kwargs):
    """Specialization of table_generic into table with as many statistics
    as will fit plus embedded slat
    diagram as suitable for supplementary material. Multiple tables are
    formed, one for each in *bas* with lines *mtd* within each table.

    """
    rowplan = ['bas', 'opt', 'mtd']
    columnplan = [
        ['l', r"""Method \& Basis Set""", '', textables.label, {}],
        ['d', 'BBI', 'SHB', colorval(0.10), {'sset': 'shb', 'dbse': 'BBI'}],
        ['d', 'BBI', 'UA', colorval(0.08), {'sset': 'ua', 'dbse': 'BBI'}],
        ['d', 'BBI', 'TT', colorval(0.10), {'sset': 'default', 'dbse': 'BBI'}],
        ['d', 'SSI', r"""$\bm{+/+}$""", colorval(0.15), {'sset': 'pospos', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+/-}$""", colorval(0.24), {'sset': 'posneg', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+}$/pl""", colorval(0.17), {'sset': 'pospolar', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+}$/al""", colorval(0.09), {'sset': 'posaliph', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{+}$/ar""", colorval(0.12), {'sset': 'posaryl', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-/-}$""", colorval(0.25), {'sset': 'negneg', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-}$/pl""", colorval(0.15), {'sset': 'negpolar', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-}$/al""", colorval(0.09), {'sset': 'negaliph', 'dbse': 'SSI'}],
        ['d', 'SSI', r"""$\bm{-}$/ar""", colorval(0.14), {'sset': 'negaryl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'pl/pl', colorval(0.12), {'sset': 'polarpolar', 'dbse': 'SSI'}],
        ['d', 'SSI', 'pl/al', colorval(0.09), {'sset': 'polaraliph', 'dbse': 'SSI'}],
        ['d', 'SSI', 'pl/ar', colorval(0.12), {'sset': 'polararyl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'al/al', colorval(0.08), {'sset': 'aliphaliph', 'dbse': 'SSI'}],
        ['d', 'SSI', 'al/ar', colorval(0.08), {'sset': 'alipharyl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'ar/ar', colorval(0.10), {'sset': 'arylaryl', 'dbse': 'SSI'}],
        ['d', 'SSI', 'TT', colorval(0.13), {'sset': 'default', 'dbse': 'SSI'}],
        ['c', r"""Iowa\footnotemark[1]""", '', textables.liliowa, {}],
    ]

    footnotes = [fnreservoir['liliowa'].format('SSI ', kwargs['ialimit'])] 
    landscape = True
    theme = 'si2bfdbmc'
    title = r"""Interaction energy (kcal/mol) {{err}} subset statistics computed with {{opt}}{0}.""".format(
                '' if kwargs['subjoin'] else r""" and {bas}""")
    return rowplan, columnplan, landscape, footnotes, title, theme

In [ ]:
def make_bfdb_Tables_S2dft(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """
    fctlplan = {
        'pbe':      (['PBE',    'PBED2',    'PBED3'],    'PBE'),
        'bp86':     (['BP86',   'BP86D2',   'BP86D3'],   'BP86'),
        'blyp':     (['BLYP',   'BLYPD2',   'BLYPD3'],   'BLYP'),
        'b97':      (['B97',    'B97D2',    'B97D3'],    'B97'),
        'pbe0':     (['PBE0',   'PBE0D2',   'PBE0D3'],   'PBE0'),
        'b3lyp':    (['B3LYP',  'B3LYPD2',  'B3LYPD3'],  'B3LYP'),
        'wpbe':     (['WPBE',               'WPBED3'],   r"""$\omega$PBE"""),
        'wb97x':    (['M052X', 'WB97XD', 'WB97XV', 'WB97MV'], r"""M05-2X, $\omega$B97X-D, $\omega$B97X-V, and $\omega$B97M-V"""),
        'b2plyp':   (['B2PLYP', 'B2PLYPD2', 'B2PLYPD3'], 'B2PLYP'),
        'pbed3':    (['PBED3',    'PBED3M',    'PBED3BJ',    'PBED3MBJ'],    'PBE-D3'),
        'bp86d3':   (['BP86D3',   'BP86D3M',   'BP86D3BJ',   'BP86D3MBJ'],   'BP86-D3'),
        'blypd3':   (['BLYPD3',   'BLYPD3M',   'BLYPD3BJ',   'BLYPD3MBJ'],   'BLYP-D3'),
        'b97d3':    (['B97D3',    'B97D3M',    'B97D3BJ',    'B97D3MBJ'],    'B97-D3'),
        'pbe0d3':   (['PBE0D3',   'PBE0D3M',   'PBE0D3BJ',   'PBE0D3MBJ'],   'PBE0-D3'),
        'b3lypd3':  (['B3LYPD3',  'B3LYPD3M',  'B3LYPD3BJ',  'B3LYPD3MBJ'],  'B3LYP-D3'),
        'wpbed3':   (['WPBED3',   'WPBED3M',   'WPBED3BJ',   'WPBED3MBJ'],   r"""$\omega$PBE-D3"""),
        'b2plypd3': (['B2PLYPD3', 'B2PLYPD3M', 'B2PLYPD3BJ', 'B2PLYPD3MBJ'], 'B2PLYP-D3'),
        'best': (['PBE0D3BJ', 'PBE0D3MBJ', 'B3LYPD3BJ', 'B3LYPD3MBJ', 'B2PLYPD3M', 'WB97XV', 'WB97MV'], 'best'),
        }
    
    for fl in fctlplan:
        tblname = 'si2' + fl
        
        def specializetheme(**kwargs):
            rowplan, columnplan, landscape, footnotes, title, theme = table_bfdb_allssetwcolor(**kwargs)
            theme += '-' + fl
            return rowplan, columnplan, landscape, footnotes, title, theme

        dbobj.table_wrapper(mtd=fctlplan[fl][0],
                            bas=['adz', 'atz', 'def2qzvp'],
                            opt=['unCP', 'CP'],
                            opttarget={'default': ['', 'dfhf', 'dfmp_dfhf']},
                            subjoin=True,                        
                            suppressblanks=True,
                            tableplan=specializetheme,
                            filename=tblname,
                            title=r"""Interaction energy (kcal/mol) {err} subset statistics by """ + fctlplan[fl][1] + """ methods.""",
                            err=tblerr,
                            failoninc=tblfailoninc,
                            xlimit=tblxlimit,
                            xlines=tblxlines,
                            ialimit=tblialimit,
                            plotpath=plotpath,
                            standalone=standalone)
        if standalone:
            !{pdflatex} {tblname} && {pdfviewer} {tblname + '.pdf'}

make_bfdb_Tables_S2dft(bfdb)

# further adjustments for suppmat inclusion
# * in best, comment out adz-uncp

## VII. Heatmaps and SAPT-tinted Iowas (Fig. 1)

In [ ]:
# isolate in a directory
subdir = 'fig_heatmaps_etc'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
bbi.plot_iowa('SAPT2P-SA-adz', view=False, xlimit=1.0)
ssi.plot_iowa('SAPT2P-SA-adz', view=False, xlimit=1.0)

In [ ]:
def natomhist(dbobj, heavyonly=False, graphicsformat=['pdf']):
    """
    
    """
    import matplotlib.pyplot as plt

    natoms = []
    rhrxn = dbobj.get_hrxn()
    for dbrxn, orxn in rhrxn.iteritems():
        orgts = orxn.rxnm['default'].keys()
        omolD = qcdb.Molecule(orgts[0].mol)
        omolD.update_geometry()
        if heavyonly:
            nat = 0
            for at in omolD.atoms:
                if at.symbol() != 'H':
                    nat += 1
            natoms.append(nat)
        else:
            natoms.append(omolD.natom())

    fig, ax1 = plt.subplots(figsize=(16, 6))
    plt.axvline(0.0, color='#cccc00')
    xmin, xmax = 0, 50
    ax1.set_xlim(xmin, xmax)
    ax1.hist(natoms, bins=50, range=(xmin, xmax), color='#2d4065', alpha=0.7)

    for ext in graphicsformat:
        savefile = 'natom_' + ('heavy_' if heavyonly else 'all_') + dbobj.dbse + '.' + ext.lower()
        plt.savefig(savefile, transparent=True, format=ext, bbox_inches='tight')
    plt.show()

In [ ]:
natomhist(ssi, heavyonly=False)

In [ ]:
def heat(dbobj, graphicsformat=['pdf']):
    """For database *dbobj*, creates a heatmap plot in formats *graphicsformat*.

    """
    import re
    import numpy as np
    import matplotlib
    import matplotlib.pyplot as plt

    aa = ['ARG', 'LYS', 'ASP', 'GLU', 'SER', 'THR', 'ASN', 'GLN', 'CYS', 'MET', 'GLY', 'ALA', 'VAL', 'ILE', 'LEU', 'PRO', 'PHE', 'TYR', 'HIE', 'TRP']
    bfdbpattern = re.compile("\d\d\d([A-Z][A-Z][A-Z])-\d\d\d([A-Z][A-Z][A-Z])-\d")

    # handle for frame, overall axis
    fig, axt = plt.subplots(figsize=(6, 6))

    axt.set_xticks(np.arange(len(aa)) + 0.3, minor=False)
    axt.set_yticks(np.arange(len(aa)) + 0.3, minor=False)
    axt.invert_yaxis()
    axt.xaxis.tick_top()
    axt.set_xticklabels(aa, minor=False, rotation=60, size='small')
    axt.set_yticklabels(aa, minor=False, size='small')
    axt.xaxis.set_tick_params(width=0, length=0)
    axt.yaxis.set_tick_params(width=0, length=0)
    
    axt.axvline(x=3.85, linewidth=5, color='k')
    axt.axvline(x=7.95, linewidth=5, color='k')
    axt.axvline(x=10.95, linewidth=5, color='k')

    axt.axhline(y=3.85, linewidth=5, color='k')
    axt.axhline(y=7.95, linewidth=5, color='k')
    axt.axhline(y=10.95, linewidth=5, color='k')

    rxns = [rxn.split('-', 1)[1] for rxn in dbobj.hrxn.keys()]

    tiles = []
    for aa1 in aa:
        print '\n', aa1,
        for aa2 in aa:
            count = 0
            for rxn in rxns:
                bfdbname = bfdbpattern.match(rxn)
                if (bfdbname.group(1) == aa1 and bfdbname.group(2) == aa2) or \
                   (bfdbname.group(2) == aa1 and bfdbname.group(1) == aa2):
                    count += 1                    
            print count,
            tiles.append(count)

    vmin, vmax = 0, max(tiles)
    cb = np.reshape(np.array(tiles), (len(aa), len(aa)))

    print 'max', vmax
    heatmap = axt.pcolor(cb, vmin=vmin, vmax=vmax, cmap=plt.cm.bone_r)

    for ext in graphicsformat:
        savefile = 'heat_' + dbobj.dbse + '.' + ext.lower()
        plt.savefig(savefile, transparent=True, format=ext, bbox_inches='tight')
    plt.show()

In [ ]:
heat(bbi)
heat(ssi)

In [ ]:
def composition_tile(db, aa1, aa2):
    """Takes dictionary *db* of label, error pairs and amino acids *aa1*
    and *aa2* and returns a square array of all errors for that amino
    acid pair, buffered by zeros.

    """
    import re
    import numpy as np
    bfdbpattern = re.compile("\d\d\d([A-Z][A-Z][A-Z])-\d\d\d([A-Z][A-Z][A-Z])-\d")

    tiles = []
    for key, val in db.items():
        bfdbname = bfdbpattern.match(key)
        if (bfdbname.group(1) == aa1 and bfdbname.group(2) == aa2) or \
           (bfdbname.group(2) == aa1 and bfdbname.group(1) == aa2):
            tiles.append(val)

    dim = int(np.ceil(np.sqrt(len(tiles))))
    pad = dim * dim - len(tiles)
    tiles += [0] * pad

    return np.reshape(np.array(tiles), (dim, dim))


def rgbiowa(dbobj, view=True, graphicsformat=['pdf']):
    """For database *dbobj*, creates a heatmap plot in formats *graphicsformat*.

    """
    import re
    import numpy as np
    import matplotlib
    import matplotlib.pyplot as plt

    
    dbdat = {} 
    rhrxn = dbobj.get_hrxn()
    for dbrxn, orxn in rhrxn.iteritems():
        rxn = dbrxn.split('-', 1)[1]
        dbdat[rxn] = orxn.color

    aa = ['ARG', 'LYS', 'ASP', 'GLU', 'SER', 'THR', 'ASN', 'GLN', 'CYS', 'MET', 'GLY', 'ALA', 'VAL', 'ILE', 'LEU', 'PRO', 'PHE', 'TYR', 'HIE', 'TRP']

    colors = [('white')] + [(plt.cm.jet(i)) for i in xrange(1,256)]
    jetwhitezero = matplotlib.colors.LinearSegmentedColormap.from_list('jetwhitezero', colors, N=256)

    # handle for frame, overall axis
    fig, axt = plt.subplots(figsize=(6, 6))

    axt.set_xticks(np.arange(len(aa)) + 0.3, minor=False)
    axt.set_yticks(np.arange(len(aa)) + 0.3, minor=False)
    axt.invert_yaxis()
    axt.xaxis.tick_top()
    axt.set_xticklabels(aa, minor=False, rotation=60, size='small')
    axt.set_yticklabels(aa, minor=False, size='small')
    axt.xaxis.set_tick_params(width=0, length=0)
    axt.yaxis.set_tick_params(width=0, length=0)
    
    xmin, xmax = 0, 1
    
    # nill spacing between 20x20 heatmaps
    plt.subplots_adjust(hspace=0.001, wspace=0.001)

    index = 1
    for aa1 in aa:
        for aa2 in aa:
            cb = composition_tile(dbdat, aa1, aa2)

            ax = matplotlib.axes.Subplot(fig, len(aa), len(aa), index)
            fig.add_subplot(ax)
            heatmap = ax.pcolor(cb, vmin=xmin, vmax=xmax, cmap=jetwhitezero)
            ax.set_xticks([])
            ax.set_yticks([])
            index += 1

    axt.axvline(x=3.85, linewidth=5, color='k')
    axt.axvline(x=7.75, linewidth=5, color='k')
    axt.axvline(x=10.65, linewidth=5, color='k')
    axt.axhline(y=3.85, linewidth=5, color='k')
    axt.axhline(y=7.75, linewidth=5, color='k')
    axt.axhline(y=10.65, linewidth=5, color='k')
    axt.set_zorder(100)

    for ext in graphicsformat:
        savefile = 'rgbiowa_' + dbobj.dbse + '.' + ext.lower()
        plt.savefig(savefile, transparent=True, format=ext, bbox_inches='tight')
    if view:
        plt.show()
    plt.close()

In [ ]:
rgbiowa(bbi, view=False)
rgbiowa(ssi, view=False)

## VIII. Threads, ternaries, & Iowas of statistical subsets (Fig. 6)

In [ ]:
# isolate in a directory
subdir = 'fig_stat_subsets'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
bbi.plot_iowa('PBE0D3BJ-dfhf_CP-adz', view=False, xlimit=1.0, sset='default')
bbi.plot_iowa('PBE0D3BJ-dfhf_CP-adz', view=False, xlimit=1.0, sset='bbi25')
ssi.plot_iowa('PBE0D3BJ-dfhf_CP-adz', view=False, xlimit=1.0, sset='default')
ssi.plot_iowa('PBE0D3BJ-dfhf_CP-adz', view=False, xlimit=1.0, sset='ssi500')
ssi.plot_iowa('PBE0D3BJ-dfhf_CP-adz', view=False, xlimit=1.0, sset='ssi100')

In [ ]:
bfdb.plot_modelchems(['PBE0D3BJ-dfhf_CP-adz', 'PBE0D3BJ-dfhf_CP-adz', 'PBE0D3BJ-dfhf_CP-adz', 'PBE0D3BJ-dfhf_CP-adz', 'PBE0D3BJ-dfhf_CP-adz'], 
                     msset=['ssi', 'ssi500', 'ssi100', 'bbi', 'bbi25'], 
                     labeled=False)

## IX. IE Distribution Thread SuppMat Tables (Tables S-3 and S-4) 

In [ ]:
# isolate in a directory
subdir = 'tbl_silver_residue'
%cd {wdir}
if not os.path.exists(subdir):
    os.makedirs(subdir)
%cd {subdir}

In [ ]:
def form_residue_subsets_from_rxnname(dbobj):
    import re
    bfdbpattern = re.compile("\d\d\d([A-Z][A-Z][A-Z])-\d\d\d([A-Z][A-Z][A-Z])-\d")
    aa = ['ARG', 'LYS',
          'ASP', 'GLU',
          'SER', 'THR', 'ASN', 'GLN',
          'CYS', 'GLY', 'MET',
          'PRO', 'ALA', 'VAL', 'ILE', 'LEU',
          'PHE', 'TYR', 'HIE', 'TRP']
    aasset = {}
    for a in aa:
        aasset[a.lower()] = {db: [] for db in dbobj.dbdict.keys()}
    
    for dbrxn in dbobj.hrxn:
        db, rxn = dbrxn.split('-', 1)
        bfdbname = bfdbpattern.match(rxn)
        aa1 = bfdbname.group(1)
        aa2 = bfdbname.group(2)
        aasset[aa1.lower()][db].append(rxn)
        if aa1 != aa2:
            aasset[aa2.lower()][db].append(rxn)
    for a in aasset:
        dbobj.add_Subset(a, aasset[a])    

In [ ]:
form_residue_subsets_from_rxnname(bbi)
form_residue_subsets_from_rxnname(ssi)

In [ ]:
def table_bfdb_residue(**kwargs):
    """Specialization of table_generic into table to showcase the count, range,
    and plot of errors. Actually used for reaction energy itself, rather than error.

    """
    print kwargs
    print kwargs['xlines']
    rowplan = ['sset']
    columnplan = [
        ['l', r"""Subset""", '', textables.label, {}],
        ['d', r"""Statistics""", 'count', textables.count, {}],
        ['d', r"""Statistics""", 'min', textables.val, {'err': 'nexe'}],
        ['d', r"""Statistics""", 'mean', textables.val, {'err': 'me'}],
        ['d', r"""Statistics""", 'max', textables.val, {'err': 'pexe'}],
        ['l', r"""IE Distribution\footnotemark[1]""", '', textables.flat, {}]
    ]

    footnotes = [r"""Benchmark interaction energies (\textit{{not}} errors). Guide lines are at {0} kcal/mol bound ($-$) and unbound ($+$).""".format(
            oxcom(kwargs['xlines']))]
    landscape = False
    theme = 'sibfdbss'
    title = r"""{{dbse}} interaction energy (kcal/mol) subset range statistics with reference {{mtd}}/{{bas}}{0}.""".format(
        '' if kwargs['subjoin'] else r""" and {bas}""")
    return rowplan, columnplan, landscape, footnotes, title, theme


def make_bfdb_Tables_Sbbiresidue(dbobj):
    """Generate the subset details suppmat Part II tables and their indices for DHDFT.

    """  
    tblname = 'sibbiss'
    dbobj.table_wrapper(mtd=['DWCCSDTF12'],
                    bas=['adz'],        
                    sset=['default', 'bbi25', 
                          'shb', 'ua', 
                          'arg', 'lys', 'asp', 'glu', 'ser', 'thr', 'asn', 'gln', 'cys', 
                              'gly', 'met', 'pro', 'ala', 'val', 'ile', 'leu', 'phe', 'tyr', 'hie', 'trp'],
                    tableplan=table_bfdb_residue,
                    benchmark='ZEROS',
                    opt=[''],
                    opttarget={'default': ['CP']},
                    # all errors computed all the time so err=[] not needed
                    failoninc=False,  # safe b/c explanatory footnotes added
                    subjoin=True,
                    xlimit=10.0,
                    xlines=[0.0, 1.0, 5.0],
                    plotpath=plotpath,
                    standalone=standalone,
                    filename=tblname)
    if standalone:
        !{pdflatex} {tblname} && {pdfviewer} {tblname + '.pdf'}

def make_bfdb_Tables_Sssiresidue(dbobj):
    tblname = 'sississ'
    dbobj.table_wrapper(mtd=['DWCCSDTF12'],
                    bas=['adz'],        
                    sset=['default', 'ssi500', 'ssi100', 
                          'arg', 'lys', 'asp', 'glu', 'ser', 'thr', 'asn', 'gln', 'cys', 
                              'met', 'pro', 'ala', 'val', 'ile', 'leu', 'phe', 'tyr', 'hie', 'trp',
                          'neutral', 'cation', 'anion',
                          'pospos', 'posneg', 'pospolar', 'posaliph', 'posaryl',
                          'negneg', 'negpolar', 'negaliph', 'negaryl',
                          'polarpolar', 'polaraliph', 'polararyl', 
                          'aliphaliph', 'alipharyl',
                          'arylaryl'],
                    tableplan=table_bfdb_residue,
                    benchmark='ZEROS',
                    opt=[''],
                    opttarget={'default': ['CP']},
                    # all errors computed all the time so err=[] not needed
                    failoninc=False,  # safe b/c explanatory footnotes added
                    subjoin=True,
                    xlimit=130.0,
                    xlines=[0.0, 10.0, 100.0],
                    plotpath=plotpath,
                    standalone=standalone,
                    filename=tblname)
    if standalone:
        !{pdflatex} {tblname} && {pdfviewer} {tblname + '.pdf'}

In [ ]:
# run from jupyter, pop up tables, fresh build figures
plotpath = 'autogen'
standalone = True

# run from jupyter, pop up tables, reuse figures
# plotpath = ''
# standalone = True

# run from jupyter, prep tables tex for paper/suppmat, reuse figures
# plotpath = '/opt/anaconda1anaconda2anaconda3/bfdb_analysis/tbl_silver_residue/'
# standalone = False

In [ ]:
make_bfdb_Tables_Sbbiresidue(bbi)
make_bfdb_Tables_Sssiresidue(ssi)

# X. Misc.

In [ ]:
ssi.analyze_modelchems(['SAPT2P-SA-adz']) #, 'B2PLYPD3-dfhf_dfmp_unCP-def2qzvp', 'B2PLYPD3M-dfhf_dfmp_unCP-def2qzvp', 'B2PLYPD3BJ-dfhf_dfmp_unCP-def2qzvp', 'B2PLYPD3MBJ-dfhf_dfmp_unCP-def2qzvp'])

In [ ]:
ssi.analyze_modelchems(['B3LYPD3-dfhf_CP-def2qzvp', 'BLYPD3-dfhf_CP-def2qzvp'], failoninc=False)

In [ ]:
ssi.plot_modelchems(['B3LYPD3BJ-dfhf_CP-adz', 'B3LYPD3BJ-dfhf_CP-atz', 'B3LYPD3BJ-dfhf_CP-def2qzvp'], sset='negneg')
ssi.plot_modelchems(['BLYP-dfhf_CP-adz', 'BLYP-dfhf_CP-def2qzvp'], sset='negneg', xlimit=8)

In [ ]:
ssi.analyze_modelchems(['PBEH3C-dfhf_unCP-def2msvp'])

In [ ]:
a = ssi.compute_statistics('GAFF--na', failoninc=False)
print('GAFF', a['SSI']['maxe'])
a = ssi.compute_statistics('CGENFF--na', failoninc=False)
print('CGENFF', a['SSI']['maxe'])
a = ssi.compute_statistics('AM1--na', failoninc=False)
print('AM1', a['SSI']['maxe'])
a = ssi.compute_statistics('PM6DH2--na', failoninc=False)
print('PM6DH2', a['SSI']['maxe'])
ssi.compute_statistics('M052X-dfhf_CP-adz', failoninc=False)

In [ ]:
print(bfdb.get_missing_reactions('GAFF--na', sset='ionblk'))
print(bfdb.get_missing_reactions('GAFF--na', sset='plslice'))
print(bfdb.get_missing_reactions('GAFF--na', sset='npblk'))
bfdb.get_missing_reactions('GAFF--na')

In [ ]:
print(bfdb.get_missing_reactions('CGENFF--na', sset='ionblk'))
print(bfdb.get_missing_reactions('CGENFF--na', sset='plslice'))
print(bfdb.get_missing_reactions('CGENFF--na', sset='npblk'))
bfdb.get_missing_reactions('CGENFF--na')

In [ ]:
print(bfdb.get_missing_reactions('AM1--na', sset='ionblk'))
print(bfdb.get_missing_reactions('AM1--na', sset='plslice'))
print(bfdb.get_missing_reactions('AM1--na', sset='npblk'))
bfdb.get_missing_reactions('AM1--na')

In [ ]:
print(bfdb.get_missing_reactions('BP86-dfhf_CP-adz', sset='ionblk'))
print(bfdb.get_missing_reactions('B97-dfhf_CP-adz', sset='ionblk'))
print(bfdb.get_missing_reactions('BLYP-dfhf_CP-adz', sset='ionblk'))
print(bfdb.get_missing_reactions('PBE-dfhf_CP-adz', sset='ionblk'))
print(bfdb.get_missing_reactions('B97-dfhf_CP-atz', sset='ionblk'))
print(bfdb.get_missing_reactions('PBE-dfhf_CP-atz', sset='ionblk'))
print(bfdb.get_missing_reactions('BP86-dfhf_CP-def2qzvp', sset='ionblk'))
print(bfdb.get_missing_reactions('B97-dfhf_CP-def2qzvp', sset='ionblk'))
print(bfdb.get_missing_reactions('BLYP-dfhf_CP-def2qzvp', sset='ionblk'))
print(bfdb.get_missing_reactions('PBE-dfhf_CP-def2qzvp', sset='ionblk'))

In [ ]:
hsg = qcdb.Database('hsg', loadfrompickle=True)
hsg.load_qcdata_byproject('bfdbmm')
ubq = qcdb.Database('ubq', loadfrompickle=True)
ubq.load_qcdata_byproject('bfdbmm')

In [ ]:
hbc = qcdb.Database('hbc6', loadfrompickle=True)
#hbc.load_qcdata_byproject('dhdft')
hbc.load_qcdata_byproject('dfitm')
nbc = qcdb.Database('nbc10', loadfrompickle=True)
#nbc.load_qcdata_byproject('dhdft')
nbc.load_qcdata_byproject('dfitm')

In [ ]:
mc = ['B3LYPD3M-dfhf_CP-adz']
mc = ['B3LYPD3-CP-adz']
#mc = ['B3LYPD3-CP-adz', 'B3LYPD3M-dfhf_CP-adz', 'B3LYPD3MBJ-dfhf_CP-adz']
dashes = ['', 
          #'BJ',
          'M', 
          #'MBJ',
         ]
#B2PLYP-D3M or -D3M(BJ)/aDZ-CP
mc = ['BLYP' + 'D3' + d + '-dfhf_CP-adz' for d in dashes]
# B2PLYP-D3M/aTZ-CP, B3LYP-D3M(BJ)/aDZ-CP, PBE0-D3M(BJ)/aDZ-CP, $\omega$B97M-V/aTZ-unCP, $\omega$B97X-V/aTZ-unCP
#BLYP-D3M/aDZ-CP;
#PBE-D3M/aTZ-unCP;
#BLYP-D3M/aDZ-CP
mc = ['BLYPD3M-dfhf_CP-adz', 'B3LYPD3MBJ-dfhf_CP-adz']
# hbc.plot_axis('Rrat', mc)
# nbc.plot_axis('Rrat', mc)
nbc.plot_axis('Rrat', mc, sset='bzbz_s')
nbc.plot_axis('Rrat', mc, sset='bzbz_t')
nbc.plot_axis('Rrat', mc, sset='bzbz_pd34')
nbc.plot_axis('Rrat', mc, sset='bzme')
nbc.plot_axis('Rrat', mc, sset='meme')
nbc.plot_axis('Rrat', mc, sset='pypy_s2')
nbc.plot_axis('Rrat', mc, sset='pypy_t3')
nbc.plot_axis('Rrat', mc, sset='bzh2s')
hbc.plot_axis('Rrat', mc, sset='faoofaoo')
hbc.plot_axis('Rrat', mc, sset='faonfaon')
hbc.plot_axis('Rrat', mc, sset='fannfann')
hbc.plot_axis('Rrat', mc, sset='faoofaon')
hbc.plot_axis('Rrat', mc, sset='faonfann')
hbc.plot_axis('Rrat', mc, sset='faoofann')

In [ ]:
# Divide SSI/BBI into old HB/MX/DD clases

def get_sset_from_color(sector, dbinstance):
    eligible = []
    for rxn, orxn in dbinstance.hrxn.iteritems():
        if sector == 'dd' and 0.000 < orxn.color and orxn.color < 0.333:
            eligible.append(rxn)
        if sector == 'mx' and 0.333 < orxn.color and orxn.color < 0.667:
            eligible.append(rxn)
        if sector == 'hb' and 0.667 < orxn.color and orxn.color < 1.000:
            eligible.append(rxn)
    return eligible

def get_hb_from_color(dbinstance):
    return get_sset_from_color('hb', dbinstance)

def get_mx_from_color(dbinstance):
    return get_sset_from_color('mx', dbinstance)

def get_dd_from_color(dbinstance):
    return get_sset_from_color('dd', dbinstance)

bfdb.add_Subset('hb', {'SSI': get_hb_from_color, 'BBI': get_hb_from_color})
bfdb.add_Subset('mx', {'SSI': get_mx_from_color, 'BBI': get_mx_from_color})
bfdb.add_Subset('dd', {'SSI': get_dd_from_color, 'BBI': get_dd_from_color})